In [19]:
%matplotlib inline
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt
import matplotlib.animation as animation
from IPython.display import clear_output
from IPython.display import HTML
import pickle
import time
# from google.colab import files

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
import numpy as np
from pathlib import Path
from torchvision import transforms
import torch
from torch import nn
import torchvision.utils as vutils

In [21]:
from utils import visual_data, load_cifar10
from networks import Generator, Discriminator, weights_init, DiscriminatorMiniBatchDiscrimination, DCGAN
from trainer import  update_params
from losses import D_loss, G_featMatch_loss, G_loss

In [22]:
!nvidia-smi

Wed Apr 15 22:07:27 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 960     Off  | 00000000:01:00.0  On |                  N/A |
| 22%   62C    P2    32W / 128W |   2198MiB /  4040MiB |      2%      Default |
+-------------------------------+----------------------+----------------------+
                                                                               
+-----------------------------------------------------------------------------+
| Processes:                                                       GPU Memory |
|  GPU  

In [23]:
# Setting up constants
# device
ngpu = 1

# single image
imageSize = 64
imageMean = (0.4923172 , 0.48307145, 0.4474483)
imageStd = (0.24041407, 0.23696952, 0.25565723)

# data loader
numWorkers = 3
batchSize = 16

# Network Arch
nc = 3 # Number of channels
nz = 100 # Latent vector
ngf = 64 # relates to the depth of feature maps carried through the generator
ndf = 64 # sets the depth of feature maps propagated through the discriminator

# Training
num_epochs = 5

# Adam Optimizer
lr = .0002
beta1 = .5

# convention of the labeling for the real and the fake datasets
## one-sided label smoothing
real_label = .9
fake_label = 0

# label smoothing
## insdead of real label=.9 give uniform between .8, 1
## insdead of fake label=0 give uniform between 0, .2
label_smoothing = True

# flip labels, with probability pFlip, flip the labels passed to the discriminator
pFlip = 0.05

# if to use last two layers' features for feature matching
double_layer = False

# whether to do minibatch normalization
miniBatchDiscrimination = True

In [24]:
# folder to store/load data
dataFolder = Path("./data")
# Decide which device to run on
device = torch.device("cuda:0" if (torch.cuda.is_available() and ngpu > 0) else "cpu")

In [25]:
torch.cuda.is_available()

True

In [26]:
cifarFolder = dataFolder/"CIFAR10"
# prepare data for loading
tsfms = transforms.Compose([
    transforms.Resize(imageSize), 
    transforms.ToTensor(),
    transforms.Normalize(imageMean, imageStd)
])
trainLoader, test_loader = load_cifar10(cifarFolder, tsfms, batchSize, numWorkers)

Files already downloaded and verified
Files already downloaded and verified


In [27]:
# init D and G network
gen = Generator(ngpu, nz, ngf, nc).to(device)
gen.apply(weights_init);

if not miniBatchDiscrimination:
    disc = Discriminator(ngpu, nc, ndf).to(device)
else: 
    disc = DiscriminatorMiniBatchDiscrimination(ngpu, nc, ndf).to(device)

disc.apply(weights_init);

In [28]:
dcgan = DCGAN(gen, disc, device, 
              real_label, fake_label, pFlip, label_smoothing, double_layer)

In [29]:
# setup optmization 
optimizerD = torch.optim.Adam(dcgan.discriminator.parameters(), lr=lr, betas=(beta1, 0.999))
# optimizerD = torch.optim.SGD(disc.parameters(), lr=0.1, momentum=0.9)
optimizerG = torch.optim.Adam(dcgan.generator.parameters(), lr=lr, betas=(beta1, 0.999))

In [30]:
# fixed noise z for viusalization of the progress of the training
fixed_noise = torch.randn(batchSize, dcgan.generator.nz, 1, 1, device=device)

# training loop 
# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
g=0
print("Starting Training Loop...")

Starting Training Loop...


In [31]:
num_epochs = 30

In [32]:
from gan import trainer_GAN
trainer = trainer_GAN(dcgan, trainLoader, optimizerD, optimizerG)
trainer.train()

[0000/3125]  Loss_D: 1.347  Loss_G: inf t= 0.156
[0050/3125]  Loss_D: 1.069  Loss_G: 0.985 t= 4.848
[0100/3125]  Loss_D: 0.742  Loss_G: 1.104 t= 4.998
[0150/3125]  Loss_D: 0.682  Loss_G: 1.059 t= 4.881
[0200/3125]  Loss_D: 0.767  Loss_G: 0.824 t= 5.000


Process Process-16:
Process Process-17:
Process Process-18:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
Traceback (most recent call last):
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in 

KeyboardInterrupt: 

  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/queues.py", line 104, in get
    if not self._poll(timeout):
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/connection.py", line 257, in poll
    return self._poll(timeout)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiprocessing/connection.py", line 414, in _poll
    r = wait([self], timeout)
  File "/home/rainbow/miniconda3/envs/dcgan_faster/lib/python3.6/multiproc

In [ ]:
len(G_losses)

In [ ]:
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses,label="G")
plt.plot(D_losses,label="D", alpha=.5)
plt.xlabel("iterations")
plt.ylabel("Loss")
plt.legend()
plt.show()

In [ ]:
trainIter = iter(trainLoader)

In [ ]:
real_batch = next(trainIter)

noise = torch.randn(64, nz, 1, 1, device=device)
fake = gen(noise).detach().cpu()
fake = vutils.make_grid(fake, padding=2, normalize=True)

# Plot the real images
plt.figure(figsize=(15,15))
plt.subplot(1,2,1)
plt.axis("off")
plt.title("Real Images")
plt.imshow(np.transpose(vutils.make_grid(real_batch[0].to(device)[:64], padding=5, normalize=True).cpu(),(1,2,0)))

# Plot the fake images from the last epoch
plt.subplot(1,2,2)
plt.axis("off")
plt.title("Fake Images")
plt.imshow(np.transpose(fake,(1,2,0)))
plt.show()